## Build a logistic regression model and show the label& Predictions

In [1]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}

Intitializing Scala interpreter ...

Spark Web UI available at http://54ad5bde17e3:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559608601502)
SparkSession available as 'spark'


import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}


In [2]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}


In [3]:
val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [4]:
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [5]:
val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [6]:
val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [7]:
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [8]:
val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET"))

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [9]:
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   train_df=train_df.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [10]:
train_df.printSchema()

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_EMPLOYED: float (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: float (nullable = true)
 |-- bureau_DAYS_CREDIT_max: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_mean: float (nullable = true)
 |-- DAYS_REGISTRATION: 

 |-- previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_OVERDUE_mean: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Construction_count_norm: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_SCOFR_count_norm: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Furniture_count_norm: float (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max: float (nullable = true)
 |-- previous_loans_NAME_GOODS_CATEGORY_Construction Materials_count_norm: float (nullable = true)
 |-- client_bureau_balance_STATUS_1_count_norm_sum: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_min_min: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_VERIF_count_norm: float (nullable 

In [11]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val df2 = assembler.transform(train_df)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f9c7497e25e0
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 281 more fields]


In [12]:
df2.printSchema()

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_EMPLOYED: float (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: float (nullable = true)
 |-- bureau_DAYS_CREDIT_max: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_mean: float (nullable = true)
 |-- DAYS_REGISTRATION: 

 |-- previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_OVERDUE_mean: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Construction_count_norm: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_SCOFR_count_norm: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Furniture_count_norm: float (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max: float (nullable = true)
 |-- previous_loans_NAME_GOODS_CATEGORY_Construction Materials_count_norm: float (nullable = true)
 |-- client_bureau_balance_STATUS_1_count_norm_sum: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_min_min: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_VERIF_count_norm: float (nullable 

In [13]:
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_9ce61d54b6d2
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 282 more fields]


In [14]:
val model = new LogisticRegression().fit(df3)
val predictions = model.transform(df3)

2019-06-04 00:37:51,843 WARN  [Executor task launch worker for task 16] memory.MemoryStore (Logging.scala:logWarning(66)) - Not enough space to cache rdd_56_1 in memory! (computed 21.7 MB so far)
2019-06-04 00:37:51,859 WARN  [Executor task launch worker for task 16] storage.BlockManager (Logging.scala:logWarning(66)) - Persisting block rdd_56_1 to disk instead.
2019-06-04 00:37:55,005 WARN  [Executor task launch worker for task 15] memory.MemoryStore (Logging.scala:logWarning(66)) - Not enough space to cache rdd_56_0 in memory! (computed 41.6 MB so far)
2019-06-04 00:37:55,010 WARN  [Executor task launch worker for task 15] storage.BlockManager (Logging.scala:logWarning(66)) - Persisting block rdd_56_0 to disk instead.
2019-06-04 00:38:19,778 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-06-04 00:38:19,779 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation fr

model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_055625adccd7, numClasses = 2, numFeatures = 281
predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 285 more fields]


In [15]:
predictions.select ("features", "label", "prediction").show()

2019-06-04 00:38:57,950 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[1.73074993654154...|  0.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|[5.26803021784871...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  0.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       1.0|
|(281,[0,1,2,3,4,5...|  0.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  0.0|       0.0|
|[0.00124580401461...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  0.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.00160539394710...|  0.0|       0.0|
|(281,[0,1,2,3,4,5...|  0.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|      

## Now we check the accuracy of the prediction:

In [16]:
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.SparkSession


In [17]:
    val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")
    val accuracy = evaluator.evaluate(predictions)
    println("Test set accuracy = " + accuracy)

Test set accuracy = 0.928218534338554


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_fbfd4ed5c449
accuracy: Double = 0.928218534338554


## We can see that the accuracy of our prediction is over 90%. Much higher than our threshold 50%

## Or we use a more intuitive way to check accuracy: We filter the amount of rows in which 'label' and 'prediction' doesn't match (in other word the rows with prediction failure)

In [24]:
predictions.count()

res10: Long = 128529


In [25]:
predictions.filter(!($"label" <=> $"prediction")).select ("features", "label", "prediction").show()

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|[5.26803021784871...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.00124580401461...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  0.0|       1.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.00415406282991...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.00992821343243...|  1.0|       0.0|
|[0.01022860966622...|  1.0|       0.0|
|(281,[0,1,2,3,4,5...|  1.0|       0.0|
+--------------------+-----+----------+
only showing top 20 rows



In [26]:
predictions.filter(!($"label" <=> $"prediction")).select ("features", "label", "prediction").count()

res12: Long = 9226


## We can see that among the 128529 rows, the prediction failure is of 9226 rows. The accuracy is over 90%